In [1]:
import numpy as np
import pandas as pd
from SARIMA import SARIMA
from sklearn.svm import SVR

In [2]:
load_df = pd.read_csv('Data/DOM/Load Actuals/Processed/Aggregated/load.csv')
load_df['lag1'] = load_df['MW'].shift()
load_df['lag2'] = load_df['MW'].shift(2)
load_df['lag3'] = load_df['MW'].shift(3)
load_df['lag4'] = load_df['MW'].shift(4)
load_df

,DATE,MW,lag1,lag2,lag3,lag4
0,2010-01-01 00:00:00,10273.0,NaN,NaN,NaN,NaN
1,2010-01-01 01:00:00,9960.0,10273.0,NaN,NaN,NaN
2,2010-01-01 02:00:00,9797.0,9960.0,10273.0,NaN,NaN
3,2010-01-01 03:00:00,9715.0,9797.0,9960.0,10273.0,NaN
4,2010-01-01 04:00:00,9851.0,9715.0,9797.0,9960.0,10273.0
...,...,...,...,...,...,...
87643,2019-12-31 19:00:00,11682.0,11954.0,11767.0,10853.0,10259.0
87644,2019-12-31 20:00:00,11443.0,11682.0,11954.0,11767.0,10853.0
87645,2019-12-31 21:00:00,11167.0,11443.0,11682.0,11954.0,11767.0
87646,2019-12-31 22:00:00,10839.0,11167.0,11443.0,11682.0,11954.0


In [3]:
weather_df = pd.read_csv('Data/DOM/Weather/Processed/Aggregated/weather.csv')
weather_df['DEW_C'] = (weather_df['DEW'] - 32) * (5.0/9)
weather_df['TMP_C'] = (weather_df['TMP'] - 32) * (5.0/9)
weather_df['RH'] = (np.exp((17.625*weather_df['DEW_C'])/(243.04+weather_df['DEW_C']))/np.exp((17.625*weather_df['TMP_C'])/(243.04+weather_df['TMP_C']))) 
weather_df.drop(['TMP', 'DEW', 'WND', 'DEW_C', 'TMP_C'], axis=1, inplace=True)
weather_df

,DATE,CR,HR,XHR,RH
0,2010-01-01 00:00:00,0.0,13.0,1.0,0.924584
1,2010-01-01 01:00:00,0.0,13.0,2.0,0.924584
2,2010-01-01 02:00:00,0.0,13.0,1.0,0.924584
3,2010-01-01 03:00:00,0.0,13.0,2.0,0.961468
4,2010-01-01 04:00:00,0.0,13.0,2.0,0.924584
...,...,...,...,...,...
87643,2019-12-31 19:00:00,0.0,4.0,0.0,0.398229
87644,2019-12-31 20:00:00,0.0,4.0,0.0,0.396591
87645,2019-12-31 21:00:00,0.0,5.0,0.0,0.411376
87646,2019-12-31 22:00:00,0.0,8.0,0.0,0.515690


In [4]:
calendar_df = pd.read_csv('Data/calendar.csv')

calendar_df

,DATE,IS_MONDAY,IS_TUESDAY,IS_WEDNESDAY,IS_THURSDAY,IS_FRIDAY,IS_SATURDAY,IS_SUNDAY,IS_JANUARY,IS_FEBRUARY,...,IS_JULY,IS_AUGUST,IS_SEPTEMBER,IS_OCTOBER,IS_NOVEMBER,IS_DECEMBER,IS_PREVDAY_HOLIDAY,IS_CURRENTDAY_HOLIDAY,IS_NEXTDAY_HOLIDAY,IS_DAYAFTER_HOLIDAY
0,2010-01-01 00:00:00,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,2010-01-01 01:00:00,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,2010-01-01 02:00:00,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,2010-01-01 03:00:00,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,2010-01-01 04:00:00,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87643,2019-12-31 19:00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
87644,2019-12-31 20:00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
87645,2019-12-31 21:00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
87646,2019-12-31 22:00:00,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [5]:
df = pd.concat([load_df,weather_df, calendar_df]).groupby('DATE').mean()

df = df[4:]
df

,MW,lag1,lag2,lag3,lag4,CR,HR,XHR,RH,IS_MONDAY,...,IS_JULY,IS_AUGUST,IS_SEPTEMBER,IS_OCTOBER,IS_NOVEMBER,IS_DECEMBER,IS_PREVDAY_HOLIDAY,IS_CURRENTDAY_HOLIDAY,IS_NEXTDAY_HOLIDAY,IS_DAYAFTER_HOLIDAY
DATE,,,,,,,,,,,,,,,,,,,,,
2010-01-01 04:00:00,9851.0,9715.0,9797.0,9960.0,10273.0,0.0,13.0,2.0,0.924584,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2010-01-01 05:00:00,10178.0,9851.0,9715.0,9797.0,9960.0,0.0,13.0,1.0,0.924584,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2010-01-01 06:00:00,10497.0,10178.0,9851.0,9715.0,9797.0,0.0,13.0,2.0,0.924584,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2010-01-01 07:00:00,10749.0,10497.0,10178.0,9851.0,9715.0,0.0,13.0,1.0,0.888800,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2010-01-01 08:00:00,11000.0,10749.0,10497.0,10178.0,9851.0,0.0,13.0,2.0,0.924256,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,11682.0,11954.0,11767.0,10853.0,10259.0,0.0,4.0,0.0,0.398229,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2019-12-31 20:00:00,11443.0,11682.0,11954.0,11767.0,10853.0,0.0,4.0,0.0,0.396591,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2019-12-31 21:00:00,11167.0,11443.0,11682.0,11954.0,11767.0,0.0,5.0,0.0,0.411376,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [6]:
x = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [7]:
from sklearn.model_selection import train_test_split

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, shuffle=False, test_size=168)

In [8]:
x_test.lag1 = np.nan
x_test.lag2 = np.nan
x_test.lag3 = np.nan
x_test.lag4 = np.nan

In [9]:
sarima_train = y_train[-672:]
sarima_test = y_test

In [11]:
import statsmodels.api as sm

sarima_model = sm.tsa.statespace.SARIMAX(sarima_train,
                                        order=(1, 1, 1),
                                        seasonal_order=(1, 1, 1, 24),
                                        enforce_stationarity=False,
                                        enforce_invertibility=False, freq='H').fit(disp=False)

/home/alic/PycharmProjects/SVR-PSO-Load-Forecaster/venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


In [12]:
pred = sarima_model.forecast(steps=168)
pred

2019-12-25 00:00:00    11145.882039
2019-12-25 01:00:00    11110.840974
2019-12-25 02:00:00    11150.354811
2019-12-25 03:00:00    11307.229161
2019-12-25 04:00:00    11664.796724
                           ...     
2019-12-31 19:00:00    13760.684415
2019-12-31 20:00:00    13642.202957
2019-12-31 21:00:00    13310.205058
2019-12-31 22:00:00    12781.209855
2019-12-31 23:00:00    12255.762152
Freq: H, Name: predicted_mean, Length: 168, dtype: float64

In [13]:
pred = pred.round()
pred

2019-12-25 00:00:00    11146.0
2019-12-25 01:00:00    11111.0
2019-12-25 02:00:00    11150.0
2019-12-25 03:00:00    11307.0
2019-12-25 04:00:00    11665.0
                        ...   
2019-12-31 19:00:00    13761.0
2019-12-31 20:00:00    13642.0
2019-12-31 21:00:00    13310.0
2019-12-31 22:00:00    12781.0
2019-12-31 23:00:00    12256.0
Freq: H, Name: predicted_mean, Length: 168, dtype: float64

In [14]:
pred = pd.DataFrame(pred)
pred.set_index(pd.to_datetime(pred.index), inplace=True)


In [15]:
x_test = pd.DataFrame(x_test)
x_test.set_index(pd.to_datetime(x_test.index), inplace=True)

x_test

,lag1,lag2,lag3,lag4,CR,HR,XHR,RH,IS_MONDAY,IS_TUESDAY,...,IS_JULY,IS_AUGUST,IS_SEPTEMBER,IS_OCTOBER,IS_NOVEMBER,IS_DECEMBER,IS_PREVDAY_HOLIDAY,IS_CURRENTDAY_HOLIDAY,IS_NEXTDAY_HOLIDAY,IS_DAYAFTER_HOLIDAY
DATE,,,,,,,,,,,,,,,,,,,,,
2019-12-25 00:00:00,NaN,NaN,NaN,NaN,0.0,10.0,0.0,0.675633,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 01:00:00,NaN,NaN,NaN,NaN,0.0,13.0,2.0,0.788691,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 02:00:00,NaN,NaN,NaN,NaN,0.0,13.0,2.0,0.787841,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 03:00:00,NaN,NaN,NaN,NaN,0.0,14.0,4.0,0.852405,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 04:00:00,NaN,NaN,NaN,NaN,0.0,15.0,4.0,0.851784,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,NaN,NaN,NaN,NaN,0.0,4.0,0.0,0.398229,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2019-12-31 20:00:00,NaN,NaN,NaN,NaN,0.0,4.0,0.0,0.396591,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2019-12-31 21:00:00,NaN,NaN,NaN,NaN,0.0,5.0,0.0,0.411376,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [16]:
from datetime import datetime
from datetime import timedelta

for index, row in x_test.iterrows():
    lag1_time = index - timedelta(hours=1)
    lag2_time = index - timedelta(hours=2)
    lag3_time = index - timedelta(hours=3)
    lag4_time = index - timedelta(hours=4)

    if lag1_time in pred.index:
        row.lag1 = pred.loc[lag1_time]

    if lag2_time in pred.index:
        row.lag2 = pred.loc[lag2_time]

    if lag3_time in pred.index:
        row.lag3 = pred.loc[lag3_time]

    if lag4_time in pred.index:
        row.lag4 = pred.loc[lag4_time]

x_test

,lag1,lag2,lag3,lag4,CR,HR,XHR,RH,IS_MONDAY,IS_TUESDAY,...,IS_JULY,IS_AUGUST,IS_SEPTEMBER,IS_OCTOBER,IS_NOVEMBER,IS_DECEMBER,IS_PREVDAY_HOLIDAY,IS_CURRENTDAY_HOLIDAY,IS_NEXTDAY_HOLIDAY,IS_DAYAFTER_HOLIDAY
DATE,,,,,,,,,,,,,,,,,,,,,
2019-12-25 00:00:00,NaN,NaN,NaN,NaN,0.0,10.0,0.0,0.675633,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 01:00:00,11146.0,NaN,NaN,NaN,0.0,13.0,2.0,0.788691,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 02:00:00,11111.0,11146.0,NaN,NaN,0.0,13.0,2.0,0.787841,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 03:00:00,11150.0,11111.0,11146.0,NaN,0.0,14.0,4.0,0.852405,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 04:00:00,11307.0,11150.0,11111.0,11146.0,0.0,15.0,4.0,0.851784,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,13813.0,13702.0,12826.0,12278.0,0.0,4.0,0.0,0.398229,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2019-12-31 20:00:00,13761.0,13813.0,13702.0,12826.0,0.0,4.0,0.0,0.396591,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2019-12-31 21:00:00,13642.0,13761.0,13813.0,13702.0,0.0,5.0,0.0,0.411376,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [17]:
y_train = pd.DataFrame(y_train)
y_train.set_index(pd.to_datetime(y_train.index), inplace=True)
y_train

,MW
DATE,
2010-01-01 04:00:00,9851.0
2010-01-01 05:00:00,10178.0
2010-01-01 06:00:00,10497.0
2010-01-01 07:00:00,10749.0
2010-01-01 08:00:00,11000.0
...,...
2019-12-24 19:00:00,11706.0
2019-12-24 20:00:00,11710.0
2019-12-24 21:00:00,11664.0


In [18]:
for index, row in x_test.iterrows():
    lag1_time = index - timedelta(hours=1)
    lag2_time = index - timedelta(hours=2)
    lag3_time = index - timedelta(hours=3)
    lag4_time = index - timedelta(hours=4)

    if np.isnan(row.lag1):
        row.lag1 = y_train.loc[lag1_time]

    if np.isnan(row.lag2):
        row.lag2 = y_train.loc[lag2_time]

    if np.isnan(row.lag3):
        row.lag3 = y_train.loc[lag3_time]

    if np.isnan(row.lag4):
        row.lag4 = y_train.loc[lag4_time]

x_test

,lag1,lag2,lag3,lag4,CR,HR,XHR,RH,IS_MONDAY,IS_TUESDAY,...,IS_JULY,IS_AUGUST,IS_SEPTEMBER,IS_OCTOBER,IS_NOVEMBER,IS_DECEMBER,IS_PREVDAY_HOLIDAY,IS_CURRENTDAY_HOLIDAY,IS_NEXTDAY_HOLIDAY,IS_DAYAFTER_HOLIDAY
DATE,,,,,,,,,,,,,,,,,,,,,
2019-12-25 00:00:00,11315.0,11519.0,11664.0,11710.0,0.0,10.0,0.0,0.675633,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 01:00:00,11146.0,11315.0,11519.0,11664.0,0.0,13.0,2.0,0.788691,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 02:00:00,11111.0,11146.0,11315.0,11519.0,0.0,13.0,2.0,0.787841,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 03:00:00,11150.0,11111.0,11146.0,11315.0,0.0,14.0,4.0,0.852405,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2019-12-25 04:00:00,11307.0,11150.0,11111.0,11146.0,0.0,15.0,4.0,0.851784,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,13813.0,13702.0,12826.0,12278.0,0.0,4.0,0.0,0.398229,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2019-12-31 20:00:00,13761.0,13813.0,13702.0,12826.0,0.0,4.0,0.0,0.396591,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2019-12-31 21:00:00,13642.0,13761.0,13813.0,13702.0,0.0,5.0,0.0,0.411376,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns)

In [20]:
#model = SVR(kernel='rbf', C=701.3536328125001, gamma=0.023333005593254352)
model = SVR(kernel='rbf', C=701.3536328125001, gamma=0.023333005593254352)
model.fit(x_train_scaled, y_train['MW'])

SVR(C=701.3536328125001, gamma=0.023333005593254352)

In [26]:
y_pred_scaled = model.predict(x_test_scaled)

In [39]:
y_test

,MW,pred
DATE,,
2019-12-25 00:00:00,11198.0,11000.383833
2019-12-25 01:00:00,11192.0,10886.434178
2019-12-25 02:00:00,11288.0,10937.096430
2019-12-25 03:00:00,11511.0,11087.450845
2019-12-25 04:00:00,11804.0,11324.634858
...,...,...
2019-12-31 19:00:00,11682.0,13397.395605
2019-12-31 20:00:00,11443.0,13229.537092
2019-12-31 21:00:00,11167.0,13004.551867


In [32]:
from sklearn.metrics import mean_absolute_percentage_error

mean_absolute_percentage_error(y_test, y_pred_scaled)
y_test = pd.DataFrame(y_test)
y_test['pred'] = y_pred_scaled
y_test['mape'] = (y_test['MW'] - y_test['pred']) / y_test['MW']
y_test

ValueError: y_true and y_pred have different number of output (2!=1)

In [40]:
y_test['mape'] = (y_test['MW'] - y_test['pred']) / y_test['MW']
y_test

,MW,pred,mape
DATE,,,
2019-12-25 00:00:00,11198.0,11000.383833,0.017647
2019-12-25 01:00:00,11192.0,10886.434178,0.027302
2019-12-25 02:00:00,11288.0,10937.096430,0.031086
2019-12-25 03:00:00,11511.0,11087.450845,0.036795
2019-12-25 04:00:00,11804.0,11324.634858,0.04061
...,...,...,...
2019-12-31 19:00:00,11682.0,13397.395605,-0.146841
2019-12-31 20:00:00,11443.0,13229.537092,-0.156125
2019-12-31 21:00:00,11167.0,13004.551867,-0.164552
